In [ ]:
# --- 5. Add Bonds from PDB Topology ---
"""
num_bonds = 0
residues = list(topology.residues())
pos_nm = [p.value_in_unit(u.nanometer) for p in positions]

print(f"Finding bonds in PDB topology...")
for bond in topology.bonds():
    atom1_idx = bond.atom1.index
    atom2_idx = bond.atom2.index
    

    pos1 = positions[atom1_idx]
    pos2 = positions[atom2_idx]
    
    delta = pos2 - pos1
    r0_dist = u.sqrt(u.sum(delta * delta)) # sqrt(dx^2 + dy^2 + dz^2)
    
    # Add this bond to the FENE force
    feneBond.addBond(atom1_idx, atom2_idx, [k_bond, R0, r0_dist])
    num_bonds += 1

if num_bonds == 0:
    print("WARNING: No bonds were found in your 'system.pdb' (no CONECT records).")
    print("The FENE force has been added but will not affect anything.")
else:
    print(f"Added {num_bonds} bonds to the FENE force.")
"""
def add_bond_to_system(idx1, idx2):
    """Calculates distance and adds bond to FENE force"""
    p1 = pos_nm[idx1]
    p2 = pos_nm[idx2]
    
    # Calculate Euclidean distance for r0 (equilibrium)
    dx, dy, dz = p1[0]-p2[0], p1[1]-p2[1], p1[2]-p2[2]
    r0_dist = u.sqrt(dx*dx + dy*dy + dz*dz)
    
    # Add Bond: Must strip units for CustomBondForce parameters
    feneBond.addBond(idx1, idx2, [k_bond,R0,r0_dist])
    return 1
for i, res in enumerate(residues):
    ca_atom = next((a for a in res.atoms() if a.name == "CA"), None)
    sc_atom = next((a for a in res.atoms() if a.name == "SC"), None)
    # 1. SIDECHAIN BOND: Connect CA(i) -> SC(i)
    if sc_atom is not None:
        num_bonds += add_bond_to_system(ca_atom.index, sc_atom.index)

    # 2. BACKBONE BOND: Connect CA(i) -> CA(i+1)
    if i < len(residues) - 1:
        next_res = residues[i+1]
        next_ca = next((a for a in next_res.atoms() if a.name == "CA"), None)
        
        if next_ca is not None:
            num_bonds += add_bond_to_system(ca_atom.index, next_ca.index)